# A new SymPy backend for vector: uniting experimental and theoretical physicists

Jim Pivarski <sup>\*</sup>, and Saransh Chopra <sup>\*</sup> <sup>+</sup> (speaker)

<sup>\*</sup>: Princeton University <sup>+</sup>: Cluster Innovation Centre, University of Delhi

[![Talk](https://img.shields.io/badge/PyHEP24-notebook_talk-blue?logo=github&logoColor=white&color=blue)](https://indico.cern.ch/event/1150631/contributions/5014393/)
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/Saransh-cpp/PyHEP24-vector-sympy-backend/HEAD?urlpath=lab/tree/talk.ipynb)

Let's start by importing the required libraries and reading a data file from `scikit-hep-testdata` -

In [1]:
import skhep_testdata
import uproot
import vector
import sympy

data = uproot.open(skhep_testdata.data_path("uproot-HZZ.root"))

We can now extract the data through the TTree using the 'Events' key -

In [2]:
branches = data['events'].arrays()

Let's take the first Muon of the second and third event as sample data -

In [3]:
px_1 = branches['Muon_Px'][1][0]
py_1 = branches['Muon_Py'][1][0]
pz_1 = branches['Muon_Pz'][1][0]
E_1 = branches['Muon_E'][1][0]

px_2 = branches['Muon_Px'][2][0]
py_2 = branches['Muon_Py'][2][0]
pz_2 = branches['Muon_Pz'][2][0]
E_2 = branches['Muon_E'][2][0]

We can now create Object type vectors and perform vector methods (such as `deltaR`) on the same -

In [4]:
muon_1_obj = vector.MomentumObject4D(px=px_1, py=py_1, pz=pz_1, E=E_1)
muon_2_obj = vector.MomentumObject4D(px=px_2, py=py_2, pz=pz_2, E=E_2)

In [5]:
muon_1_obj.deltaR(muon_2_obj)

1.3067924381658622

SymPy vectors do not require numerical coordinates, instead, we can create symbolic containers using SymPy and pass them directly into `VectorSympy`/`MomentumSympy` contructors -

In [6]:
px_1, py_1, pz_1, E_1 = sympy.symbols("px_1 py_1 pz_1 E_1", real=True)
px_2, py_2, pz_2, E_2 = sympy.symbols("px_2 py_2 pz_2 E_2", real=True)

In [7]:
muon_1_sympy = vector.MomentumSympy4D(px=px_1, py=py_1, pz=pz_1, E=E_1)
muon_2_sympy = vector.MomentumSympy4D(px=px_2, py=py_2, pz=pz_2, E=E_2)

Similar to other backends, one can perform vector methods (such as `deltaR`) on SymPy vectors, and the method will work automatically on SymPy symbols (`Symbol`) or expressions (`Expr`). Jupyter turns on SymPy's latex printing by default, but one can toggle it in terminal using `sympy.init_session`. 

In [8]:
muon_1_sympy.deltaR(muon_2_sympy)

sqrt((Mod(atan2(py_1, px_1) - atan2(py_2, px_2) + pi, 2*pi) - pi)**2 + (asinh(pz_1/sqrt(px_1**2 + py_1**2)) - asinh(pz_2/sqrt(px_2**2 + py_2**2)))**2)

Given that all the computation results are SymPy expressions, any SymPy functionality can be used on them. Here we use the `subs` function to substitute values in place of the symbols -

In [9]:
muon_1_sympy.deltaR(muon_2_sympy).subs(
    {
        px_1: muon_1_obj.px,
        py_1: muon_1_obj.py,
        pz_1: muon_1_obj.pz,
        E_1: muon_1_obj.E
    }
)

sqrt((0.753814 - asinh(pz_2/sqrt(px_2**2 + py_2**2)))**2 + (Mod(1.53735 - 1.0*atan2(py_2, px_2), 2*pi) - pi)**2)

We can similarly substitute all the values and obtain a numerical result -

In [10]:
muon_1_sympy.deltaR(muon_2_sympy).subs(
    {
        px_1: muon_1_obj.px,
        py_1: muon_1_obj.py,
        pz_1: muon_1_obj.pz,
        E_1: muon_1_obj.E,
        px_2: muon_2_obj.px,
        py_2: muon_2_obj.py,
        pz_2: muon_2_obj.pz,
        E_2: muon_2_obj.E,
    }
)

sqrt(0.299083 + (1.95474 - pi)**2)

To evalute the numerical result, one can use SymPy's `evalf` method -

In [11]:
muon_1_sympy.deltaR(muon_2_sympy).subs(
    {
        px_1: muon_1_obj.px,
        py_1: muon_1_obj.py,
        pz_1: muon_1_obj.pz,
        E_1: muon_1_obj.E,
        px_2: muon_2_obj.px,
        py_2: muon_2_obj.py,
        pz_2: muon_2_obj.pz,
        E_2: muon_2_obj.E,
    }
).evalf()

1.30679233752151

and voila, the results from the Object backend and SymPy backend match!

One can experiment with larger symbolic expressions as well. Let's boost a vector and obtain the `px` coordinate using the Object backend -

In [12]:
v = vector.MomentumObject4D(pt=1, phi=2, eta=3, M=10)

In [13]:
v.boost(v.to_beta3()).px

-1.1810297606283302

The exact similar operation can be carried out using the SymPy backend. Let's create a SymPy vector with SymPy symbols -

In [14]:
pt, phi, eta, M = sympy.symbols("pt phi eta M", real=True)
v = vector.MomentumSympy4D(pt=pt, phi=phi, eta=eta, M=M)

The operations will now work on the symbols instead of numerical values -

In [15]:
v.to_beta3()

MomentumSympy3D(pt=pt/sqrt(M**2 + 0.25*pt**2*(1 + exp(-2*eta))**2*exp(2*eta)), phi=phi, eta=eta)

In [16]:
v.boost(v.to_beta3()).px

pt**3*sin(phi)**2*cos(phi)/((1 + 1/sqrt(-pt**2*sin(phi)**2/(M**2 + 0.25*pt**2*(1 + exp(-2*eta))**2*exp(2*eta)) - pt**2*cos(phi)**2/(M**2 + 0.25*pt**2*(1 + exp(-2*eta))**2*exp(2*eta)) - pt**2*sinh(eta)**2/(M**2 + 0.25*pt**2*(1 + exp(-2*eta))**2*exp(2*eta)) + 1))*(M**2 + 0.25*pt**2*(1 + exp(-2*eta))**2*exp(2*eta))*(-pt**2*sin(phi)**2/(M**2 + 0.25*pt**2*(1 + exp(-2*eta))**2*exp(2*eta)) - pt**2*cos(phi)**2/(M**2 + 0.25*pt**2*(1 + exp(-2*eta))**2*exp(2*eta)) - pt**2*sinh(eta)**2/(M**2 + 0.25*pt**2*(1 + exp(-2*eta))**2*exp(2*eta)) + 1)) + pt**3*cos(phi)*sinh(eta)**2/((1 + 1/sqrt(-pt**2*sin(phi)**2/(M**2 + 0.25*pt**2*(1 + exp(-2*eta))**2*exp(2*eta)) - pt**2*cos(phi)**2/(M**2 + 0.25*pt**2*(1 + exp(-2*eta))**2*exp(2*eta)) - pt**2*sinh(eta)**2/(M**2 + 0.25*pt**2*(1 + exp(-2*eta))**2*exp(2*eta)) + 1))*(M**2 + 0.25*pt**2*(1 + exp(-2*eta))**2*exp(2*eta))*(-pt**2*sin(phi)**2/(M**2 + 0.25*pt**2*(1 + exp(-2*eta))**2*exp(2*eta)) - pt**2*cos(phi)**2/(M**2 + 0.25*pt**2*(1 + exp(-2*eta))**2*exp(2*eta)) - 

Given that the result itself is a SymPy expression, we can further use SymPy's `simplify` method to reduce or simplify the result. The simplification step might take some time!

In [17]:
v.boost(v.to_beta3()).px.simplify()

pt*(sqrt(M**2 + pt**2*cosh(eta)**2)*(sqrt((1.0*M**2*exp(2*eta) + 0.25*pt**2*exp(4*eta) - 1.0*pt**2*exp(2*eta)*sinh(eta)**2 - 0.5*pt**2*exp(2*eta) + 0.25*pt**2)/(M**2*exp(2*eta) + 0.25*pt**2*(exp(2*eta) + 1)**2)) + 1)*(1.0*M**2*exp(2*eta) + 0.25*pt**2*exp(4*eta) - 1.0*pt**2*exp(2*eta)*sinh(eta)**2 - 0.5*pt**2*exp(2*eta) + 0.25*pt**2)*exp(eta) + sqrt(1.0*M**2*exp(2*eta) + 0.25*pt**2*exp(4*eta) - 1.0*pt**2*exp(2*eta)*sinh(eta)**2 - 0.5*pt**2*exp(2*eta) + 0.25*pt**2)*(1.0*M**2*sqrt((1.0*M**2*exp(2*eta) + 0.25*pt**2*exp(4*eta) - 1.0*pt**2*exp(2*eta)*sinh(eta)**2 - 0.5*pt**2*exp(2*eta) + 0.25*pt**2)/(M**2*exp(2*eta) + 0.25*pt**2*exp(4*eta) + 0.5*pt**2*exp(2*eta) + 0.25*pt**2))*exp(2*eta) + 1.0*M**2*exp(2*eta) + 0.25*pt**2*sqrt((1.0*M**2*exp(2*eta) + 0.25*pt**2*exp(4*eta) - 1.0*pt**2*exp(2*eta)*sinh(eta)**2 - 0.5*pt**2*exp(2*eta) + 0.25*pt**2)/(M**2*exp(2*eta) + 0.25*pt**2*exp(4*eta) + 0.5*pt**2*exp(2*eta) + 0.25*pt**2))*exp(4*eta) + 0.5*pt**2*sqrt((1.0*M**2*exp(2*eta) + 0.25*pt**2*exp(4*eta)

Finally, we substitute the values and evaluate the SymPy expression to validate the result -

In [18]:
values = {pt: 1, phi: 2, eta: 3, M: 10}
v.boost(v.to_beta3()).px.subs(values)

sqrt(cos(2)**2 + sin(2)**2 + 100 + sinh(3)**2)*cos(2)/(10*sqrt(1 + 0.0025*(exp(-6) + 1)**2*exp(6))*sqrt(-sinh(3)**2/(100 + 0.25*(exp(-6) + 1)**2*exp(6)) - sin(2)**2/(100 + 0.25*(exp(-6) + 1)**2*exp(6)) - cos(2)**2/(100 + 0.25*(exp(-6) + 1)**2*exp(6)) + 1)) + (cos(2)**2/((1 + 1/sqrt(-sinh(3)**2/(100 + 0.25*(exp(-6) + 1)**2*exp(6)) - sin(2)**2/(100 + 0.25*(exp(-6) + 1)**2*exp(6)) - cos(2)**2/(100 + 0.25*(exp(-6) + 1)**2*exp(6)) + 1))*(100 + 0.25*(exp(-6) + 1)**2*exp(6))*(-sinh(3)**2/(100 + 0.25*(exp(-6) + 1)**2*exp(6)) - sin(2)**2/(100 + 0.25*(exp(-6) + 1)**2*exp(6)) - cos(2)**2/(100 + 0.25*(exp(-6) + 1)**2*exp(6)) + 1)) + 1)*cos(2) + cos(2)*sinh(3)**2/((1 + 1/sqrt(-sinh(3)**2/(100 + 0.25*(exp(-6) + 1)**2*exp(6)) - sin(2)**2/(100 + 0.25*(exp(-6) + 1)**2*exp(6)) - cos(2)**2/(100 + 0.25*(exp(-6) + 1)**2*exp(6)) + 1))*(100 + 0.25*(exp(-6) + 1)**2*exp(6))*(-sinh(3)**2/(100 + 0.25*(exp(-6) + 1)**2*exp(6)) - sin(2)**2/(100 + 0.25*(exp(-6) + 1)**2*exp(6)) - cos(2)**2/(100 + 0.25*(exp(-6) + 1)**

In [19]:
v.boost(v.to_beta3()).px.subs(values).evalf()

-1.18102976062833